In [1]:
#정형 데이터 => 특성별로 정리된 데이터 / 앙상블 학습 알고리즘이 가장 뛰어남 (랜덤 포레스트가 대표적)
#비정형 데이터 => 텍스트, 사진 
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy() # 분류(샘플을 몇 개의 클래스 중 하나로) / 회귀(임의의 숫자 예측)

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [2]:
# 랜덤 포레스트 => 결정트리를 랜덤하게 만들고 각 결정트리의 예측을 사용해 최종 예측 
# 트리를 만들기 위해 데이터를 랜덤하게 만든다. 1개 뽑고 다시 넣고 또 뽑고 => 부트스트랩 샘플(훈련 세트의 크기와 같게 만든다)
# 전체 특성개수의 제곱근 만큼만 사용한다. ex) 4개라면 2개만 
# 기본적으로 100개의 결정 트리 훈련
# 랜덤하게 선택한 특성과 샘플이기 때문에 훈련 세트에 과대적합 방지
# rf는 결정 트리의 앙상블 이기 때문에 DecisionTreeClassifier가 제공하는 중요한 매개변수 모두 제공
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1) # 교차 검증 수행
# return_train_score=True 검증 점수 뿐만 아니라 훈련 세트에 대한 점수도 같이 반환 (기본값 False) / 과대적합 판단 용이

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [3]:
rf.fit(train_input, train_target)
print(rf.feature_importances_) # 특성 중요도
#결정 트리와 비교하여 각 특성의 중요도가 조금씩 상승 => 과대적합 줄이고 일반화 성능 높임

[0.23167441 0.50039841 0.26792718]


In [4]:
#부트스트랩 샘플에 포함되지 않고 남는 샘플 => OOB(Out Of Bag)
# OOB로 부트스트랩 샘플로 훈련한 결정 트리 평가 => 검증 세트 역할 
# oob_score=True 각 결정트리의 OOB점수 평균하여 출력 
# OOB점수를 사용하면 교차 검증을 대신 할 수 있다 => 훈련세트에 더 많은 샘플 사용가능
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)

rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


In [5]:
# 엑스트라 트리즈 / 랜덤 포레스트와 비슷한 앙상블 학습 모델 , 부트스트랩 사용X , 특성 랜덤 선택하여 빠르다.
# et는 결정 트리의 앙상블 이기 때문에 DecisionTreeClassifier가 제공하는 중요한 매개변수 모두 제공
# 엑스트라 트리가 사용하는 결정 트리가 splitter=random인 결정 트리. (결정 트리파트 참고) => 하나의 결정 트리에서 특성 무작위로 분할 
# 빠르다!
from sklearn.ensemble import ExtraTreesClassifier # 엑스트라 트리의 회귀 버전 => ExtraTreesRegressor 

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1) # 교차 검증

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [6]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


In [16]:
# 그레이티언트 부스팅 / 깊이가 얕은 결정 트리 사용하여 이전 트리의 오차 보완하는 방식으로 앙상블
# 기본적으로 깊이가 3인 결정트리 100개 => 과대적합에 강하고 높은 일반화 성능
# 경사 하강법 사용하여 트리를 앙상블에 추가 
# 결정 트리를 계속 추가하며 가장 낮은곳 향해 이동, 손실 함수의 낮은 곳으로 천천히 조금씩 => 깊이가 얕은 트리 사용
# 학습률 매개변수로 속도 조절 
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
#과대적합이 거의 안됨
# 학습률 증가시키고 트리 개수 늘리면 성능 향상 / learning_rate 기본값 0.1
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
# 그레이디언트 부스팅이 랜덤 포레스트 보다 조금 더 높은 성능 하지만 순서대로 트리 추가 때문에 느리다.

0.8881086892152563 0.8720430147331015


In [8]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [17]:
gb = GradientBoostingClassifier(n_estimators=1000, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9695497652678313 0.8799315169911898


In [18]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.1648851  0.66776294 0.16735196]
